In [1]:
import numpy as np
import pandas as pd
import iql
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib inline
import seaborn as sns
%reload_ext ishbook
import MySQLdb as mdb

import statsmodels.stats.weightstats as sms
import datetime as dt

# Job title data

## What are the other job titles that people looking for Data Science jobs also look for?

https://go.indeed.com/IQLMXYGPH3

In [81]:
query = """
FROM 
    organic 2018-01-01 2018-02-01
WHERE 
    ctk IN (
        FROM organic 2018-01-01 2018-02-01
        WHERE (country="us") (category="tech") (q="data scientist")
        GROUP BY ctk
        )
    country="us"
    page=1
    q!=""
    category="tech"
GROUP BY 
    q[1000000]
SELECT 
    DISTINCT(ctk),
    COUNT()
"""

In [82]:
queries_from_ds = %iql2 {query}

In [83]:
queries_from_ds.columns = ['query', 'ctks', 'count']

In [84]:
# Need to clean the queries by removing white spaces and quotes 

In [85]:
queries_from_ds['query'] = queries_from_ds['query'].str.replace('"', '')
queries_from_ds['query'] = queries_from_ds['query'].map(lambda x: x.strip())

In [95]:
queries_from_ds = queries_from_ds.groupby('query').sum()
queries_from_ds.sort_values('count', ascending=False, inplace=True)

In [96]:
queries_from_ds.reset_index(inplace=True)

Top 10 queries besides "data scientist"

In [105]:
queries_from_ds[0:11]

,query,ctks,count
0,data scientist,24829,547996
1,data analyst,3954,91979
2,machine learning,1253,45002
3,data science,1513,37963
4,software engineer,718,32253
5,python,827,22185
6,data engineer,592,20544
7,statistics,584,11823
8,sas,313,11738
9,business analyst,777,11400


## What are the normtitles that jobseekers who search for "data scientist" end up clicking on?

In [109]:
query = """
FROM organic  2018-01-01 2018-02-01
WHERE 
    ctk IN (
        FROM organic 2018-01-01 2018-02-01
        WHERE (country="us") (category="tech") (q="data scientist")
        GROUP BY ctk
        )
    country="us"
    category="tech"
    clicked=1
GROUP BY 
    normTitle
SELECT 
    COUNT()
"""

In [110]:
titles_clicked_from_ds = %iql2 {query}

In [111]:
titles_clicked_from_ds.columns = ['normTitle', 'COUNT']

In [112]:
titles_clicked_from_ds.sort_values('COUNT', inplace=True, ascending=False)

#### Top 20 titles clicked

In [145]:
titles_clicked_from_ds[0:20]

,normTitle,COUNT
325,data scientist,120935
832,machine learning engineer,12883
1536,software engineer,11489
312,data analyst,9737
329,data warehouse engineer,6763
559,full stack developer,3537
139,business intelligence analyst,3501
364,development operations engineer,2914
357,developer,2859
695,java developer,2585


## What are the skills associated with each job title?

https://go.indeed.com/IQLH8E973P

In [122]:
query = """
FROM organic  2018-01-01 2018-02-01
WHERE 
    ctk IN (
        FROM organic 2018-01-01 2018-02-01
        WHERE (country="us") (category="tech") (q="data scientist")
        GROUP BY ctk
        )
    country="us"
    normTitle="{normtitle}"
    category="tech"
    clicked=1
GROUP BY 
    verifiedExtractedEntity
SELECT 
    COUNT()
"""

In [133]:
normtitle_list = list(titles_clicked_from_ds['normTitle'][0:7])

skills_df = pd.DataFrame()
for normtitle in normtitle_list:
    query_fmt = query.format(normtitle=normtitle)
    
    temp_df = %%iql2 {query_fmt}
    
    temp_df.columns = ['verifiedExtractedEntity', 'COUNT']
    temp_df['normtitle'] = normtitle
    
    skills_df = skills_df.append(temp_df)
    print(normtitle)

data scientist
machine learning engineer
software engineer
data analyst
data warehouse engineer
full stack developer
business intelligence analyst


In [134]:
# Keep only the skills and not benefit/certification
skills_only_df = skills_df[skills_df['verifiedExtractedEntity'].str.contains('skill')]

In [135]:
skills_only_df.sort_values(['normtitle', 'COUNT'], ascending=False, inplace=True)

In [ ]:
top10_skills_df = skills_only_df.groupby('normtitle').head(10)

In [143]:
top10_skills_df[top10_skills_df['normtitle']=='data scientist']

,verifiedExtractedEntity,COUNT,normtitle
175,skill/programming/python,91984,data scientist
141,skill/machine_learning,89421,data scientist
176,skill/programming/r,78071,data scientist
83,skill/data_science,64898,data scientist
82,skill/data_mining,41536,data scientist
115,skill/framework/hadoop,39183,data scientist
70,skill/big_data,36411,data scientist
122,skill/framework/spark,34690,data scientist
179,skill/programming/sas,32028,data scientist
165,skill/programming/java,31714,data scientist


In [136]:
top10_skills_df[top10_skills_df['normtitle']=='machine learning engineer']

,verifiedExtractedEntity,COUNT,normtitle
87,skill/machine_learning,12473,machine learning engineer
114,skill/programming/python,9378,machine learning engineer
97,skill/programming/c_or_c_plus_plus,5360,machine learning engineer
105,skill/programming/java,5251,machine learning engineer
34,skill/artificial_intelligence,4767,machine learning engineer
89,skill/natural_language_processing,4408,machine learning engineer
74,skill/framework/spark,4296,machine learning engineer
75,skill/framework/tensorflow,3681,machine learning engineer
115,skill/programming/r,3445,machine learning engineer
69,skill/framework/hadoop,3013,machine learning engineer


In [137]:
top10_skills_df[top10_skills_df['normtitle']=='software engineer']

,verifiedExtractedEntity,COUNT,normtitle
160,skill/programming/java,5877,software engineer
170,skill/programming/python,4770,software engineer
150,skill/programming/c_or_c_plus_plus,4341,software engineer
161,skill/programming/javascript,4145,software engineer
49,skill/agile_software_development,3885,software engineer
154,skill/programming/dot_net,3046,software engineer
126,skill/html,2249,software engineer
144,skill/platform/amazon_web_service,2115,software engineer
190,skill/programming_tools/git,2029,software engineer
135,skill/machine_learning,1872,software engineer


In [138]:
top10_skills_df[top10_skills_df['normtitle']=='data analyst']

,verifiedExtractedEntity,COUNT,normtitle
135,skill/programming/python,3185,data analyst
67,skill/data_warehouse,2971,data analyst
162,skill/programming_tools/tableau,2598,data analyst
136,skill/programming/r,2532,data analyst
74,skill/database/microsoft_sql_server,2484,data analyst
59,skill/business_intelligence,1943,data analyst
138,skill/programming/sas,1861,data analyst
65,skill/data_mining,1624,data analyst
56,skill/big_data,1523,data analyst
110,skill/machine_learning,1435,data analyst


In [139]:
top10_skills_df[top10_skills_df['normtitle']=='data warehouse engineer']

,verifiedExtractedEntity,COUNT,normtitle
133,skill/programming/python,4566,data warehouse engineer
58,skill/data_warehouse,3705,data warehouse engineer
84,skill/framework/hadoop,3319,data warehouse engineer
45,skill/big_data,3195,data warehouse engineer
123,skill/programming/java,3054,data warehouse engineer
89,skill/framework/spark,3017,data warehouse engineer
110,skill/platform/amazon_web_service,2623,data warehouse engineer
103,skill/machine_learning,2049,data warehouse engineer
138,skill/programming/scala,1936,data warehouse engineer
38,skill/agile_software_development,1796,data warehouse engineer


In [140]:
top10_skills_df[top10_skills_df['normtitle']=='business intelligence analyst']

,verifiedExtractedEntity,COUNT,normtitle
44,skill/business_intelligence,3142,business intelligence analyst
120,skill/programming_tools/tableau,1555,business intelligence analyst
52,skill/data_warehouse,1380,business intelligence analyst
58,skill/database/microsoft_sql_server,1019,business intelligence analyst
104,skill/programming/python,710,business intelligence analyst
105,skill/programming/r,695,business intelligence analyst
107,skill/programming/sas,587,business intelligence analyst
61,skill/database/oracle,567,business intelligence analyst
50,skill/data_mining,553,business intelligence analyst
123,skill/project_management,484,business intelligence analyst


In [141]:
field_of_study_df = skills_df[skills_df['verifiedExtractedEntity'].str.contains('field_of_study')]

In [142]:
field_of_study_df.sort_values(['normtitle', 'COUNT'], ascending=False)

,verifiedExtractedEntity,COUNT,normtitle
33,field_of_study/computer_science,193,software engineer
32,field_of_study/computer_engineering,26,software engineer
35,field_of_study/engineering,7,software engineer
36,field_of_study/mathematics,4,software engineer
34,field_of_study/economics,2,software engineer
31,field_of_study/business,1,software engineer
37,field_of_study/physics,1,software engineer
38,field_of_study/statistics,1,software engineer
23,field_of_study/computer_science,147,machine learning engineer
27,field_of_study/mathematics,26,machine learning engineer
